In [1]:
import torch
import numpy as np

import datasets
from tqdm.notebook import tqdm

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding
)
from copy import deepcopy

from GlobEnc.src.modeling.modeling_bert_saliency import BertForSequenceClassification
from GlobEnc.src.modeling.modeling_roberta_saliency import RobertaForSequenceClassification

import time

2023-01-20 20:02:46.869988: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-20 20:02:47.038772: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-20 20:02:47.625224: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/modaresi/.conda/envs/globenc-venv/lib/
2023-01-20 20:02:47.625315: W tensorflow/

In [2]:
models_dir = "/home/modaresi/projects/globenc_analysis/outputs/models"
outputs_dir = "/home/modaresi/projects/globenc_analysis/outputs/integrated_saliencies"

# configs = {
#     "mnli": {
#         "model_path": lambda step: f"{models_dir}/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-{step}/",
#         "output_file_path": lambda step, agg_t, bl_t: f"{outputs_dir}/mnli_bert-base-uncased_0001_SEED0042_checkpoint-{step}-{agg_t}-{bl_t}.npy",
#         "hf_ds": "mnli",
#     },
#     "sst2": {
#         "model_path": lambda step: f"{models_dir}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-{step}/",
#         "output_file_path": lambda step, agg_t, bl_t: f"{outputs_dir}/sst2_bert-base-uncased_0001_SEED0042_checkpoint-{step}-{agg_t}-{bl_t}.npy",
#         "hf_ds": "sst2",
#     }
# }

MODEL_DATASET_SET_PARTS = [
#     (f"{models_dir}/output_cola_bert-base-uncased_0001_SEED0042/checkpoint-1340", "cola", "validation"),
#     (f"{models_dir}/output_sst2_bert-base-uncased_0001_SEED0042/checkpoint-10525", "sst2", "train", ["sentence"]),
#     (f"{models_dir}/output_hatexplain_bert-base-uncased_0001_SEED0042/checkpoint-2405", "hatexplain", "train", ["text"]),
#     (f"{models_dir}/output_sst2_bert-large-uncased_0001_SEED0042/checkpoint-10525", "sst2", "validation"),
#     (f"{models_dir}/output_mrpc_bert-base-uncased_0001_SEED0042/checkpoint-575", "mrpc", "validation"),
#     (f"{models_dir}/output_qnli_bert-base-uncased_0001_SEED0042/checkpoint-16370", "qnli", "train", ["question", "sentence"]),
#     (f"{models_dir}/output_mnli_bert-base-uncased_0001_SEED0042/checkpoint-61360", "mnli", "train", ["premise", "hypothesis"]),
    (f"WillHeld/roberta-base-sst2", "sst2", "validation", ["sentence"]),
    (f"WillHeld/roberta-base-mnli", "mnli", "validation_matched", ["premise", "hypothesis"]),
]

# CONFIG_NAME = "mnli"
# CONFIG = configs[CONFIG_NAME]
MAX_LENGTH = 128
BATCH_SIZE = 32
SEED = 42
# DATA_SECTION = "validation_matched"
# BASELINE_TYPES = ["ALL_PAD", "ALL_ZERO"]
BASELINE_TYPES = ["ALL_ZERO"]
AGGREGATION_TYPES = ["NORM", "SUM"]
# AGGREGATION_TYPES = ["NORM"]
ALPHA_STEPS = 11
ALPHAS = np.linspace(0, 1, ALPHA_STEPS)
# STEP = 10525
# STEP = 61360

In [ ]:
# original_ds = datasets.load_dataset("glue", CONFIG["hf_ds"])[DATA_SECTION]
# original_ds

In [3]:
# tokenizer = AutoTokenizer.from_pretrained(
#     "bert-base-uncased",
#     use_fast=True,
# )

def _get_preprocessing_function(
    sentence1_key: str, 
    sentence2_key: str = None, 
    label_to_id: dict = None):

    def preprocess_function(examples):
        # Tokenize the texts
        args = (
            (examples[sentence1_key],) if sentence2_key is None else (examples[sentence1_key], examples[sentence2_key])
        )
        result = tokenizer(*args, padding=False, max_length=MAX_LENGTH, truncation=True)

        # Map labels to IDs (not necessary for GLUE tasks)
        if label_to_id is not None and "label" in examples:
            result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
        return result
    
    return preprocess_function

# def hatexplain_preprocess_function(examples):
#     def mode(lst):
#         return(max(set(lst), key=lst.count))
#     examples["text"] = " ".join(str(v) for v in examples["post_tokens"])
# #     print(examples["annotators"])
#     labels = [mode(val["label"]) for val in examples["annotators"]]
#     args = (
#         (examples["text"],)
#     )
#     result = tokenizer(*args, padding=False, max_length=MAX_LENGTH, truncation=True)
# #     print(result)
# #     result = dict(result)
# #     print("here")
# #     print()
#     print(result)
#     result["label"] = labels
    
#     return result

def aggregate_hatexplain(example):
    def mode(lst):
        return max(set(lst), key=lst.count)
    example["label"] = mode(example["annotators"]["label"])
    example["text"] = " ".join(example["post_tokens"])
    return example
    

# preprocess_function = _get_preprocessing_function(sentence1_key="premise", sentence2_key="hypothesis")
# preprocess_function = _get_preprocessing_function(sentence1_key="sentence")
# train_ds = original_ds.map(preprocess_function, batched=True)

In [ ]:
# if BASELINE_TYPE == "ALL_PAD":
#     train_ds_base_line = train_ds.to_dict()
#     for idx, val in enumerate(train_ds_base_line['input_ids']):
#         input_ids = [0 for i in range(len(val))]
# #         SEP_tokens = np.where(np.array(row['input_ids'])==102)[0]
#         input_ids[0] = 101
#         input_ids[-1] = 102
# #         input_ids[SEP_tokens] = 102
#         train_ds_base_line['input_ids'][idx] = input_ids
        
#     train_ds_base_line = datasets.Dataset.from_dict(train_ds_base_line)
    
#     train_ds_base_line.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
#     collator = DataCollatorWithPadding(tokenizer, True, MAX_LENGTH, return_tensors="pt")
#     baseline_dataloader = torch.utils.data.DataLoader(train_ds_base_line, batch_size=BATCH_SIZE, collate_fn=collator)

In [ ]:
# train_ds.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
# collator = DataCollatorWithPadding(tokenizer, True, MAX_LENGTH, return_tensors="pt")
# dataloader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, collate_fn=collator)
# dataset_size = len(train_ds)
# steps = int(np.ceil(dataset_size / BATCH_SIZE))
# num_labels = len(set(original_ds['label']))

In [ ]:
# model = BertForSequenceClassification.from_pretrained(CONFIG["model_path"](STEP))
# model.to(torch.device("cuda:0"))
# model.eval()

# # all_sals = torch.zeros(size=(dataset_size, MAX_LENGTH)).cuda()
# all_sals = np.zeros(shape=(dataset_size, MAX_LENGTH))
# it = iter(dataloader)
# if BASELINE_TYPE == "ALL_PAD":
#     baseline_it = iter(baseline_dataloader)

# for i in tqdm(range(steps)):
#     batch = next(it)
#     batch = {k: v.to(torch.device('cuda:0')) for k, v in batch.items()}
#     labels = batch['labels']
#     output = model(**batch, output_hidden_states=True)
#     x_reps = output.hidden_states[0]
#     output.hidden_states[0].retain_grad()
#     logits = output.logits
#     target_class_l_sum = torch.gather(logits, 1, labels.unsqueeze(-1)).sum()
#     target_class_l_sum.backward()
    
#     gradients = output.hidden_states[0].grad
#     model.zero_grad()
    
#     if BASELINE_TYPE == "ALL_PAD":
#         baseline_batch = next(baseline_it)
#         baseline_batch = {k: v.to(torch.device('cuda:0')) for k, v in baseline_batch.items()}
#         labels = batch['labels']
#         baseline_output = model(**baseline_batch, output_hidden_states=True)
#         x_baseline_reps = baseline_output.hidden_states[0]
#         baseline_output.hidden_states[0].retain_grad()
#         baseline_logits = baseline_output.logits
#         target_class_l_sum = torch.gather(baseline_logits, 1, labels.unsqueeze(-1)).sum()
#         target_class_l_sum.backward()
        
#         gradients += baseline_output.hidden_states[0].grad
#         model.zero_grad()
        
#     for idx, alpha in enumerate(ALPHAS):
#         if alpha == 0.0 or alpha == 1.0:
#             continue
        
        
#         if BASELINE_TYPE == "ALL_PAD":
#             x_alpha = x_baseline_reps + alpha * (x_reps - x_baseline_reps)
#         elif BASELINE_TYPE == "ALL_ZERO":
#             x_alpha = alpha * x_reps
        
#         inputs = x_alpha.detach()
#         inputs.requires_grad = True
#         labels = batch['labels']
#         output = model(inputs_embeds=inputs, labels=labels, output_hidden_states=True)
#         output.hidden_states[0].retain_grad()
#         logits = output.logits
#         target_class_l_sum = torch.gather(logits, 1, labels.unsqueeze(-1)).sum()
#         target_class_l_sum.backward()
        
#         gradients += output.hidden_states[0].grad
        
#         model.zero_grad()
        
#     if BASELINE_TYPE == "ALL_PAD":
#         integrated_saliency =  gradients * (x_reps - x_baseline_reps)
#     elif BASELINE_TYPE == "ALL_ZERO":
#         integrated_saliency =  gradients * x_reps
        
#     if AGGREGATION_TYPE == "SUM":
#         saliencies = torch.sum(integrated_saliency, dim=-1).detach().cpu()
#     elif AGGREGATION_TYPE == "NORM":
#         saliencies = torch.norm(integrated_saliency, dim=-1).detach().cpu()
    
#     length = saliencies.size()[1]
#     all_sals[i*BATCH_SIZE:(i+1)*BATCH_SIZE, :length] = saliencies

In [ ]:
# np.save(CONFIG["output_file_path"](STEP, AGGREGATION_TYPE, BASELINE_TYPE), all_sals)

In [4]:
def get_integrated_saliencies(dataloader, baseline_dataloader, baseline_type, steps, model, aggregation_type,
                             prediction_based=True):
    all_sals_norm_aggregated = np.zeros(shape=(dataset_size, MAX_LENGTH))
    all_sals_sum_aggregated = np.zeros(shape=(dataset_size, MAX_LENGTH))
    it = iter(dataloader)
    if baseline_type == "ALL_PAD":
        baseline_it = iter(baseline_dataloader)

    for i in tqdm(range(steps)):
        batch = next(it)
        batch = {k: v.to(torch.device('cuda:0')) for k, v in batch.items()}
        labels = batch['labels']
        output = model(**batch, output_hidden_states=True)
        x_reps = output.hidden_states[0]
        output.hidden_states[0].retain_grad()
        logits = output.logits
        if prediction_based:
            target_class_l_sum = torch.gather(logits, 1, torch.argmax(logits, dim=-1).unsqueeze(-1)).sum()
        else:
            target_class_l_sum = torch.gather(logits, 1, labels.unsqueeze(-1)).sum()
        target_class_l_sum.backward()

        gradients = output.hidden_states[0].grad
        predictions = torch.zeros(len(logits)).cuda()
        for idx, val in enumerate(logits):
            predictions[idx] = torch.argmax(val)
#         predictions = torch.argmax(logits.detach(), dim=-1).detach()
        predictions = predictions.type(torch.int64)
        model.zero_grad()

        if baseline_type == "ALL_PAD":
            baseline_batch = next(baseline_it)
            baseline_batch = {k: v.to(torch.device('cuda:0')) for k, v in baseline_batch.items()}
            labels = batch['labels']
            baseline_output = model(**baseline_batch, output_hidden_states=True)
            x_baseline_reps = baseline_output.hidden_states[0]
            baseline_output.hidden_states[0].retain_grad()
            baseline_logits = baseline_output.logits
            if prediction_based:
#                 target_class_l_sum = torch.gather(baseline_logits, 1, torch.argmax(baseline_logits, dim=-1).unsqueeze(-1)).sum()
                target_class_l_sum = torch.gather(baseline_logits, 1, predictions.unsqueeze(-1)).sum()
            else:
                target_class_l_sum = torch.gather(baseline_logits, 1, labels.unsqueeze(-1)).sum()
            target_class_l_sum.backward()

            gradients += baseline_output.hidden_states[0].grad
            model.zero_grad()

        for idx, alpha in enumerate(ALPHAS):
            if alpha == 0.0 or alpha == 1.0:
                continue


            if baseline_type == "ALL_PAD":
                x_alpha = x_baseline_reps + alpha * (x_reps - x_baseline_reps)
            elif baseline_type == "ALL_ZERO":
                x_alpha = alpha * x_reps

            inputs = x_alpha.detach()
            inputs.requires_grad = True
            labels = batch['labels']
            output = model(inputs_embeds=inputs, labels=labels, output_hidden_states=True)
            output.hidden_states[0].retain_grad()
            logits = output.logits
            if prediction_based:
#                 target_class_l_sum = torch.gather(logits, 1, torch.argmax(logits, dim=-1).unsqueeze(-1)).sum()
                target_class_l_sum = torch.gather(logits, 1, predictions.unsqueeze(-1)).sum()
            else:
                target_class_l_sum = torch.gather(logits, 1, labels.unsqueeze(-1)).sum()
            target_class_l_sum.backward()

            gradients += output.hidden_states[0].grad

            model.zero_grad()

        if baseline_type == "ALL_PAD":
            integrated_saliency =  gradients * (x_reps - x_baseline_reps)
        elif baseline_type == "ALL_ZERO":
            integrated_saliency =  gradients * x_reps

#         if aggregation_type == "SUM":
#             saliencies = torch.sum(integrated_saliency, dim=-1).detach().cpu()
#         elif aggregation_type == "NORM":
#             saliencies = torch.norm(integrated_saliency, dim=-1).detach().cpu()
        saliencies_norm_aggregated = torch.norm(integrated_saliency, dim=-1).detach().cpu()
        saliencies_sum_aggregated = torch.sum(integrated_saliency, dim=-1).detach().cpu()

        length = saliencies_norm_aggregated.size()[1]
        all_sals_norm_aggregated[i*BATCH_SIZE:(i+1)*BATCH_SIZE, :length] = saliencies_norm_aggregated
        all_sals_sum_aggregated[i*BATCH_SIZE:(i+1)*BATCH_SIZE, :length] = saliencies_sum_aggregated
    
    return all_sals_norm_aggregated, all_sals_sum_aggregated

In [5]:
time_reports = dict()

# tokenizer = AutoTokenizer.from_pretrained(
#     "bert-base-uncased",
#     use_fast=True,
# )

tokenizer = AutoTokenizer.from_pretrained(
    "roberta-base",
    use_fast=True,
)

for model_dataset_set_parts in MODEL_DATASET_SET_PARTS:
    model_checkpoint, task_name, set_of_data, sample_parts = model_dataset_set_parts
#     model = BertForSequenceClassification.from_pretrained(model_checkpoint)
    model = RobertaForSequenceClassification.from_pretrained(model_checkpoint)
    model.to(torch.device("cuda:0"))
    model.eval()
    
    if task_name == "hatexplain":
        original_ds = datasets.load_dataset(task_name)[set_of_data].map(aggregate_hatexplain)
        preprocess_function = _get_preprocessing_function(sentence1_key=sample_parts[0])
        train_ds = original_ds.map(preprocess_function, batched=True)
    else:
        original_ds = datasets.load_dataset("glue", task_name)[set_of_data]
        if len(sample_parts) == 1:
            preprocess_function = _get_preprocessing_function(sentence1_key=sample_parts[0])
        elif len(sample_parts) == 2:
            preprocess_function = _get_preprocessing_function(sentence1_key=sample_parts[0], sentence2_key=sample_parts[1])
        train_ds = original_ds.map(preprocess_function, batched=True)
    
#     train_ds.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
    train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    collator = DataCollatorWithPadding(tokenizer, True, MAX_LENGTH, return_tensors="pt")
    dataloader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, collate_fn=collator)
    dataset_size = len(train_ds)
    steps = int(np.ceil(dataset_size / BATCH_SIZE))
    num_labels = len(set(original_ds['label']))
    
    for bl_t in BASELINE_TYPES:
#         for agg_t in AGGREGATION_TYPES:
#             for gradient_base in ["prediction_based", "label_based"]:
        for gradient_base in ["prediction_based"]:
            t1 = time.time()
            if bl_t == "ALL_PAD":
                train_ds_base_line = train_ds.to_dict()
                for idx, val in enumerate(train_ds_base_line['input_ids']):
                    input_ids = [0 for i in range(len(val))]
                    input_ids[0] = 101
                    input_ids[-1] = 102
                    train_ds_base_line['input_ids'][idx] = input_ids

                train_ds_base_line = datasets.Dataset.from_dict(train_ds_base_line)

                train_ds_base_line.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
                collator = DataCollatorWithPadding(tokenizer, True, MAX_LENGTH, return_tensors="pt")
                baseline_dataloader = torch.utils.data.DataLoader(train_ds_base_line, batch_size=BATCH_SIZE, collate_fn=collator)

            t2 = time.time()
            saliencies_norm_aggregated, saliencies_sum_aggregated = get_integrated_saliencies(
                dataloader=dataloader,
                baseline_dataloader=baseline_dataloader if bl_t == "ALL_PAD" else None,
                baseline_type=bl_t,
                steps=steps,
                model=model,
                aggregation_type=AGGREGATION_TYPES[0],
                prediction_based=(gradient_base=="prediction_based")
            )
            t3 = time.time()

            time_reports[f"{task_name}-{set_of_data}-{bl_t}-{AGGREGATION_TYPES[0]}-{gradient_base}"] = (t3 - t1, t3 - t2)

#             file_name_norm_aggregated = f"[{task_name}]_[{set_of_data}]_[{'-'.join(model_checkpoint.split('/')[-2:])}]_[IG_{bl_t}_{AGGREGATION_TYPES[0]}_{gradient_base}]"
#             file_name_sum_aggregated = f"[{task_name}]_[{set_of_data}]_[{'-'.join(model_checkpoint.split('/')[-2:])}]_[IG_{bl_t}_{AGGREGATION_TYPES[1]}_{gradient_base}]"
            file_name_norm_aggregated = f"[{task_name}]_[{set_of_data}]_[{model_checkpoint.split('/')[-1]}]_[IG_{bl_t}_{AGGREGATION_TYPES[0]}_{gradient_base}]"
            file_name_sum_aggregated = f"[{task_name}]_[{set_of_data}]_[{model_checkpoint.split('/')[-1]}]_[IG_{bl_t}_{AGGREGATION_TYPES[1]}_{gradient_base}]"
                      
            np.save(f"{outputs_dir}/{file_name_norm_aggregated}.npy", saliencies_norm_aggregated)
            np.save(f"{outputs_dir}/{file_name_sum_aggregated}.npy", saliencies_sum_aggregated)

Using the latest cached version of the module from /opt/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Sat Sep  3 17:51:06 2022) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
Reusing dataset glue (/opt/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Parameter 'function'=<function _get_preprocessing_function.<locals>.preprocess_function at 0x7f7f0f2cd710> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/28 [00:00<?, ?it/s]

/home/modaresi/.conda/envs/globenc-venv/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2280: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  "`max_length` is ignored when `padding`=`True` and there is no truncation strategy. "
Reusing dataset glue (/opt/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/307 [00:00<?, ?it/s]

In [10]:
print(time_reports)

{'sst2-validation-ALL_ZERO-NORM-prediction_based': (215.85352873802185, 215.85351538658142), 'qnli-validation-ALL_ZERO-NORM-prediction_based': (2784.8672778606415, 2784.867276906967), 'mnli-validation_matched-ALL_ZERO-NORM-prediction_based': (4357.458532810211, 4357.458531618118), 'hatexplain-validation-ALL_ZERO-NORM-prediction_based': (625.162024974823, 625.1620240211487)}
